<h1>Install and Import Dependencies

In [82]:
!pip install torch torchvision torchaudio


ERROR: Operation cancelled by user


In [6]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [44]:
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 35.9 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,660 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,318 kB]
Hit:11 https://ppa.launchpadcontent.net/ub

In [57]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

<h1> Instantiate Model

In [58]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
#

<h1> Encode and Calculate Sentiment

In [59]:
tokens = tokenizer.encode('I loved this, absolutely the best', return_tensors='pt')

In [60]:
result = model(tokens)

In [61]:
result.logits

tensor([[-2.3882, -2.5314, -1.2469,  1.1947,  4.1058]],
       grad_fn=<AddmmBackward0>)

In [62]:
int(torch.argmax(result.logits))+1

5

<h1> COLLECT REVIEWS

In [63]:
# Set up headless Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize driver
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.youtube.com/watch?v=tGv7CUutzqU')

# Wait for comments to load
time.sleep(5)

# Scroll down to load comments
driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
time.sleep(5)

# Get comments
comment_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
comments = [comment.text for comment in comment_elements]

In [52]:
r = requests.get('https://www.youtube.com/watch?v=tGv7CUutzqU')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap')
results = soup.find_all('span', {'class':regex})
reviews = [result.text for result in results]


In [65]:
comments

['You were absolute OG when you try to sing "She said hello" at 0:44 >,<',
 "2024 I'm here about you  2:39",
 'About You by The 1975 is such a sentimental and heartbreaking way of remembering past people. realization hits us that we are never bound to forget. no matter how much we want to completely erase memories of people, we couldn’t help but to remember them in all ways. they remain on everything—even and most especially on the ones that are in need of loving and caring. they leave a part of their existence within us, making it harder to get rid of them. by that, we truly are a shared fragment of other people. maybe, no matter how terrible things were between us and them, we couldn’t help but to always have them. they will always shelter a space in our hearts and minds as wounds, scratches, and grief to forever live with. (2)',
 'I love the girl who name is Shin Phone Myat Every time I listen to this song, I always remember her. This song is our song. It reminds me of the memories 

<h1> Load Reviews into DataFrame and Score

In [74]:
import numpy as np
import pandas as pd

In [75]:
df = pd.DataFrame(np.array(comments), columns=['comments'])

In [78]:
def sentiment_score(comments):
    tokens = tokenizer.encode(comments, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [79]:
sentiment_score(df['comments'].iloc[1])

1

In [80]:
df['sentiment'] = df['comments'].apply(lambda x: sentiment_score(x[:512]))

In [81]:
df

,comments,sentiment
0,"You were absolute OG when you try to sing ""She...",1
1,2024 I'm here about you 2:39,1
2,About You by The 1975 is such a sentimental an...,5
3,I love the girl who name is Shin Phone Myat Ev...,5
4,“i miss you on the train\ni miss you in the mo...,1
5,My favoriteeee,5
6,2024 everyone??,1
7,and suddenly we’re 16 again,5
8,"I miss you, my nicole. (can someone like this ...",1
9,"you're still my favorite , even though 4 years...",5


In [30]:
df['review'].iloc[3]

"Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!"